In [16]:
import getpass
import requests
import json

# Securely collect PassiveSSH credentials
# Note: These credentials are not stored in the notebook for security
print("Setting up PassiveSSH API authentication...")
PASSIVE_SSH_USER = getpass.getpass("Enter your PassiveSSH Username: ")
PASSIVE_SSH_KEY = getpass.getpass("Enter your PassiveSSH API Key: ")

# Create a persistent session with authentication
# This session will be reused for all API calls
pssh = requests.Session()
pssh.auth = (PASSIVE_SSH_USER, PASSIVE_SSH_KEY)

print("✓ Authentication configured successfully!")
print("Ready to query PassiveSSH API endpoints.")

Setting up PassiveSSH API authentication...


Enter your PassiveSSH Username:  ········
Enter your PassiveSSH API Key:  ········


✓ Authentication configured successfully!
Ready to query PassiveSSH API endpoints.


In [21]:
banner = 'SSH-2.0-GIfQrwMLPdNQT99'
hosts = set()

print(f"Searching for hosts using SSH banner: '{banner}'")

try:
    response = pssh.get(f"https://pssh.circl.lu/banner/hosts/{banner}")
    response.raise_for_status()
    data = response.json()
    
    # Analyze the results
    if 'hosts' in data and data['hosts']:
        for host in data['hosts'][:5]:
            hosts.add(host)
    else:
        print("No hosts found using this specific banner")
        
except requests.exceptions.RequestException as e:
    print(f"Error querying banner hosts: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

host_details = {}

# Analyse hosts
for host in hosts:
    response = pssh.get(f"https://pssh.circl.lu/host/ssh/{host}")
    response.raise_for_status()
    data = response.json()

    host_details[host] = {'fingerprints': []}

    if data:
        # Check for banner
        if 'banner' in data:
            host_details[host]['banner'] = data['banner']
            
        # Check for fingerprints
        if 'keys' in data:
            for key in data["keys"]:
                host_details[host]['fingerprints'].append(key['fingerprint'])
            
        # Check for timestamps
        if 'first_seen' in data:
            host_details[host]['first_seen'] = data['first_seen']
        if 'last_seen' in data:
            host_details[host]['last_seen'] = data['last_seen']
            
# Analyse hosts history
for host in hosts:
    response = pssh.get(f"https://pssh.circl.lu/host/history/{host}")
    response.raise_for_status()
    data = response.json()

    timestamps = []
    
    for timestamp in list(data["history"].keys()):
            timestamps.append(timestamp)
    
    if timestamps:
        host_details[host]['ts_first'] = min(timestamps)
        host_details[host]['ts_last'] = max(timestamps)
        host_details[host]['ts_count'] = len(timestamps)

# Analyse fingerprints
for host in hosts:
    for fingerprint in host_details[host]['fingerprints']:
        data = pssh.get(f"https://pssh.circl.lu/fingerprint/all/{fingerprint}").json()

        associated_hosts = []
        
        for associated_host in data["hosts"]:
                associated_hosts.append(associated_host)
        
        if associated_hosts:
            host_details[host]['associated_hosts'] = associated_hosts

print(json.dumps(host_details, indent=4))

Searching for hosts using SSH banner: 'SSH-2.0-GIfQrwMLPdNQT99'
{
    "216.75.130.11": {
        "fingerprints": [
            "f2:90:ec:c6:49:fd:fc:98:70:d8:60:db:47:97:09:6a",
            "9a:ea:5b:86:51:70:34:25:0c:e6:ec:99:89:8f:6e:36"
        ],
        "banner": [
            "SSH-2.0-GIfQrwMLPdNQT99"
        ],
        "first_seen": "20210203",
        "last_seen": "20210203",
        "ts_first": "1612315016",
        "ts_last": "1612315016",
        "ts_count": 1,
        "associated_hosts": [
            "216.75.130.11"
        ]
    }
}
